# Introduction
This project is a demo of a dbt workflow for data ingestion, using a finance data ETL pipeline as an example. It emphasizes the importance of performing sanity checks on input data to prevent errors or invalid data from propagating through downstream stages of the data pipeline.

In [1]:
# ```{mermaid}
# flowchart LR
#    empty["."] --> DBT["Container: dbt-core 
#        plugin: dbt-
#   DBT-pyspark[Hive]"] --> |Generate SQL with 'dbt seed', 'dbt compile'| T(Container: Spark-Thrift)
#   T -->|Query using the generated SQL| Storage[(Container: Postgre Hive metadata store)]

# ```

In [2]:
import pandas as pd
pd.read_csv("/usr/app/demo/dbt_spark_demo_prj/seeds/transactions/raw__transactions.csv")


,id,amount,currency,status
0,1,100.00,USD,ACTIVE
1,2,200.01,SGD,ACTIVE
2,3,150.02,EUR,ACTIVE
3,4,120.03,USD,INACTIVE
4,5,180.04,SGD,INACTIVE
5,6,-300.05,JPY,ACTIVE
6,7,300.99,SUD,ACTIVE


## Project file structure

In [3]:
!tree -L 2 --sort="name" /usr/app | grep -v ".*\.toml$" | grep -v ".*\.md$" | grep -v ".*files$"

/usr/app
├── dbt
│   ├── dbt.Dockerfile
│   ├── dbt_spark_demo_prj
│   ├── docker-compose.yml
│   ├── logs
│   └── requirements.txt
├── dbt-spark
│   ├── dagger
│   ├── dbt
│   ├── docker
│   ├── docker-compose.yml
│   ├── scripts
│   ├── test.env.example
│   └── tests
├── demo
│   ├── dbt-spark-demo.html
│   ├── dbt-spark-demo.ipynb
│   ├── dbt-spark-demo_files
│   ├── dbt_spark_demo_prj
│   ├── generate-documentation.sh
│   ├── profiles.yml
│   └── results
├── docs
│   └── index.html
├── index.html
└── run-demo.sh



# Environment setup

## Docker
### Shared network for docker containers
```bash
source ./docker-common.sh
```

## Install jupyter notebook for running this demo in notebook 
This notebook is executed using jupyer-notebook kernel (http://127.0.0.1:8888) of the loaded custom-dbt container. The container is loaded using command below
```bash
jupyter notebook --NotebookApp.token='' --NotebookApp.password='' --NotebookApp.disable_check_xsrf=True --allow-root --ip=0.0.0.0 --port=8888 --no-browser
```
use allow-root for demo purpose only, not for production due to cyber security


In [4]:
%%bash
export TZ='Asia/Singapore'
date
which python
pwd

Wed Jul  2 22:53:00 +08 2025


/usr/local/bin/python


/usr/app/dbt


### Connection to read table in Hive meta-data store

## Variables

In [5]:
from dotenv import load_dotenv
import os
!pwd

!cd ../
!pwd

load_dotenv()
spark_thrift_service_name=os.getenv('SPARK_CONTAINER_SERVICE_NAME')
dbt_project_dir=os.getenv('DBT_PROJECT_DIR')
dbt_project_name=os.getenv('DBT_PROJECT_NAME')
print(spark_thrift_service_name)
print(dbt_project_dir)
print(dbt_project_name)

/usr/app/dbt


/usr/app/dbt


dbt-spark3-thrift
/usr/app/dbt/dbt_spark_demo_prj
dbt_spark_demo_prj


# Start demo

In [6]:
filepath="/usr/app/demo/results/test_start.txt"
!echo start test at > $filepath 
!export TZ='Asia/Singapore'; date >> $filepath

## Helper codes

In [7]:
from pyhive import hive
import pandas as pd
from sqlalchemy.engine import create_engine

def check_table(host=spark_thrift_service_name, message_to_be_printed=""):
    # with hive.Connection(host=spark_thrift_service_name, port=10000, 
    #                     #    username='', 
    #                     database='default') as conn:

    #     # Query all tables in the default database
    #     df = pd.read_sql("SHOW TABLES", conn)

    #     # Display tables
    #     display(df)
    print(message_to_be_printed)
    engine = create_engine(f'hive://{spark_thrift_service_name}:10000/default')
    df = pd.read_sql("SHOW TABLES;", engine)
    # display(df)
    display(df)

def show_table_details(table_name, database_name='default',
                        # host=spark_thrift_service_name,
                        port=10000):
    # sqlAlchemy api fail here due to "sqlAlchemy relection": TODO investigate root cause
    # engine = create_engine(f'hive://{spark_thrift_service_name}:10000/default')
    # df = pd.read_sql(f"select * from {table_name} limit 10", engine)

    with hive.Connection(host=spark_thrift_service_name, port=port, database=database_name) as conn:
        df = pd.read_sql(f"select * from {database_name}.{table_name}", conn)

        display(df)

    
check_table()
print("------------")
# show_table_details("transactions")


,namespace,tableName,isTemporary
0,default,raw__transactions,False
1,default,transactions,False


------------


## DBT 

In [8]:
!dbt --version

Core:
  - installed: 1.10.2
  - latest:    1.10.2 - Up to date!

Plugins:
  - spark: 1.9.2 - Up to date!




## Verify DBT setup

### profiles.yml
```yml
formal_verification_prj_name:
  outputs:
    spark:
      type: spark
      method: thrift
      host: dbt-spark3-thrift
      schema: default
      connect_timeout: 30
      authentication: NONE
  target: spark

```

In [9]:
!cat ~/.dbt/profiles.yml

send_anonymous_usage_stats: false

dbt_spark_demo_prj:
  outputs:
    spark:
      type: spark
      method: thrift
      host: dbt-spark3-thrift
      port: 10000
      schema: default
      connect_timeout: 30
      authentication: NONE
  target: spark


### dbt_project.yml
```yml
name: 'formal_verification_prj_name'
version: '1.0.0'

# This setting configures which "profile" dbt uses for this project.
profile: 'formal_verification_prj_name'

# These configurations specify where dbt should look for different types of files.
# The `model-paths` config, for example, states that models in this project can be
# found in the "models/" directory. You probably won't need to change these!
model-paths: ["models"]
analysis-paths: ["analyses"]
test-paths: ["tests"]
seed-paths: ["seeds"]
macro-paths: ["macros"]
snapshot-paths: ["snapshots"]

clean-targets:         # directories to be removed by `dbt clean`
  - "target"
  - "dbt_packages"

models:
  formal_verification_prj_name:
    # +write_json: false
    +materialized: incremental
    +file_format: parquet    

```

In [10]:
!cat /usr/app/dbt/dbt_spark_demo_prj/dbt_project.yml


# Name your project! Project names should contain only lowercase characters
# and underscores. A good package name should reflect your organization's
# name or the intended use of these models
name: 'dbt_spark_demo_prj'
version: '1.0.0'

# This setting configures which "profile" dbt uses for this project.
profile: 'dbt_spark_demo_prj'

# These configurations specify where dbt should look for different types of files.
# The `model-paths` config, for example, states that models in this project can be
# found in the "models/" directory. You probably won't need to change these!
model-paths: ["models"]
analysis-paths: ["analyses"]
test-paths: ["tests"]
seed-paths: ["seeds"]
macro-paths: ["macros"]
snapshot-paths: ["snapshots"]

clean-targets:         # directories to be removed by `dbt clean`
  - "target"
  - "dbt_packages"


# Configuring models
# Full documentation: https://docs.getdbt.com/docs/configuring-models

# In this example config, we tell dbt to build all models in the example/


### dbt debug

In [11]:
!dbt debug --project-dir /usr/app/dbt/dbt_spark_demo_prj

#to print detailed messages for debugging 
#!dbt debug --debug --project-dir /usr/app/dbt/dbt_spark_demo_prj

14:53:12  Running with dbt=1.10.2


14:53:12  dbt version: 1.10.2
14:53:12  python version: 3.11.2
14:53:12  python path: /usr/local/bin/python3
14:53:12  os info: Linux-5.15.167.4-microsoft-standard-WSL2-x86_64-with-glibc2.31


14:53:12  Using profiles dir at /root/.dbt
14:53:12  Using profiles.yml file at /root/.dbt/profiles.yml
14:53:12  Using dbt_project.yml file at /usr/app/dbt/dbt_spark_demo_prj/dbt_project.yml
14:53:12  adapter type: spark
14:53:12  adapter version: 1.9.2


14:53:12  Configuration:
14:53:12    profiles.yml file [OK found and valid]
14:53:12    dbt_project.yml file [OK found and valid]
14:53:12  Required dependencies:


14:53:12   - git [OK found]

14:53:12  Connection:
14:53:12    host: dbt-spark3-thrift
14:53:12    port: 10000
14:53:12    cluster: None
14:53:12    endpoint: None
14:53:12    schema: default
14:53:12    organization: 0
14:53:12  Registered adapter: spark=1.9.2


14:53:12    Connection test: [OK connection ok]

14:53:12  All checks passed!


## dbt command comparisons
| Command           | Purpose                                  | What it does                                                             | Output                                    |
| ----------------- | ---------------------------------------- | ------------------------------------------------------------------------ | ----------------------------------------- |
| **`dbt compile`** | Prepares SQL code for execution          | Converts Jinja + macros into raw SQL; writes compiled files to `target/` | Compiled SQL files (no warehouse changes) |
| **`dbt run`**     | Builds models (tables/views)             | Executes compiled SQL to materialize models in the data warehouse        | Tables/views in the warehouse             |
| **`dbt test`**    | Validates data quality assumptions       | Runs data tests (e.g., `not_null`, `unique`, custom) on warehouse data   | Test pass/fail results                    |
| **`dbt seed`**    | Loads static CSV data into the warehouse | Uploads CSV files from the `seeds/` directory into tables                 | Tables containing seed data               |


# Prepare SQL

## Read dbt input csv 

* read seeds/transactions.csv, 
* dbt generate raw__transaction.csv
* dbt create table default.raw_transaction at the database 

### Verify database before the operation

In [12]:
!dbt run-operation drop_table --args '{"table_name": "default.raw__transactions"}'  
!dbt run-operation drop_table --args '{"table_name": "default.transactions"}' 

14:53:17  Running with dbt=1.10.2


14:53:18  Registered adapter: spark=1.9.2


14:53:19  Found 1 seed, 1 model, 1 test, 478 macros, 3 unit tests


14:53:25  Running with dbt=1.10.2


14:53:25  Registered adapter: spark=1.9.2


14:53:26  Found 1 seed, 1 model, 1 test, 478 macros, 3 unit tests


In [13]:
check_table(message_to_be_printed="Database table before the operation")
!dbt seed --log-level info --project-dir $dbt_project_dir
check_table(message_to_be_printed="\nDatabase table after the operation")

Database table before the operation


,namespace,tableName,isTemporary


14:53:31  Running with dbt=1.10.2


14:53:31  Registered adapter: spark=1.9.2


14:53:32  Found 1 seed, 1 model, 1 test, 478 macros, 3 unit tests
14:53:32  
14:53:32  Concurrency: 1 threads (target='spark')
14:53:32  


...

14:53:34  
14:53:34  Finished running 1 seed in 0 hours 0 minutes and 1.68 seconds (1.68s).


14:53:34  
14:53:34  Completed successfully


14:53:34  
14:53:34  Done. PASS=1 WARN=0 ERROR=0 SKIP=0 NO-OP=0 TOTAL=1



Database table after the operation


,namespace,tableName,isTemporary
0,default,raw__transactions,False


## dbt compile

#### removed target/compiled/

In [14]:
!rm -rf dbt_spark_demo_prj/target/compiled/
!ls dbt_spark_demo_prj/target/compile

ls: cannot access 'dbt_spark_demo_prj/target/compile': No such file or directory


In [15]:
!ls $dbt_project_dir

dbt_project.yml  logs  macros  models  seeds  target  tests


In [16]:
!dbt compile --log-level info --project-dir $dbt_project_dir

14:53:41  Running with dbt=1.10.2


14:53:42  Registered adapter: spark=1.9.2


14:53:42  Found 1 seed, 1 model, 1 test, 478 macros, 3 unit tests
14:53:42  
14:53:42  Concurrency: 1 threads (target='spark')
14:53:42  


In [17]:
!ls /usr/app/dbt/dbt_spark_demo_prj/target/compiled/dbt_spark_demo_prj

models


## Build your models (dbt run)

## Delete database table prior dbt run that will create the table

In [18]:
check_table(message_to_be_printed="before")
!dbt run-operation drop_table --args '{"table_name": "default.transactions"}'
check_table(message_to_be_printed="\nafter")

before


,namespace,tableName,isTemporary
0,default,raw__transactions,False


14:53:48  Running with dbt=1.10.2


14:53:49  Registered adapter: spark=1.9.2


14:53:50  Found 1 seed, 1 model, 1 test, 478 macros, 3 unit tests



after


,namespace,tableName,isTemporary
0,default,raw__transactions,False


In [19]:
!rm -r $dbt_project_dir/target/run
!dbt run-operation drop_table --args '{"table_name": "default.transactions"}' 
check_table(message_to_be_printed="before")

!dbt run --log-level info --project-dir $dbt_project_dir

print("\nAfter")
!tree $dbt_project_dir/target/run
check_table(message_to_be_printed="after")

14:53:55  Running with dbt=1.10.2


14:53:56  Registered adapter: spark=1.9.2


14:53:57  Found 1 seed, 1 model, 1 test, 478 macros, 3 unit tests


before


,namespace,tableName,isTemporary
0,default,raw__transactions,False


...

14:54:05  
14:54:05  Completed successfully
14:54:05  
14:54:05  Done. PASS=1 WARN=0 ERROR=0 SKIP=0 NO-OP=0 TOTAL=1



After


/usr/app/dbt/dbt_spark_demo_prj/target/run
└── dbt_spark_demo_prj
    └── models
        └── transactions
            └── transactions.sql

3 directories, 1 file


after


,namespace,tableName,isTemporary
0,default,raw__transactions,False
1,default,transactions,False


## test model using dbt

#### test name and its test vectors

In [20]:
!cat dbt_spark_demo_prj/tests/unit/test_transactions.yml

# tests/unit/test_transactions.yml

version: 2
unit_tests:
  - name: test_currency_usd
    model: transactions
    given:
      - input: ref('raw__transactions')
        rows:
          - {id: 1, amount: 100, currency: 'USD', status: 'ACTIVE'}
    expect:
      rows:
        - {id: 1, amount_usd: 100, currency: 'USD', branch_marker: "USD_BRANCH"}

  - name: test_currency_sgd
    model: transactions
    given:
      - input: ref('raw__transactions')
        rows:
          - {id: 2, amount: 100, currency: 'SGD', status: 'ACTIVE'}
    expect:
      rows:
        - {id: 2, amount_usd: 74, branch_marker: "SGD_BRANCH"}

  - name: test_inactive
    model: transactions
    given:
      - input: ref('raw__transactions')
        rows:
          - {id: 2, amount: 100, currency: 'SGD', status: 'INACTIVE'}
    expect: 
      rows: []

  # - name: test_decimal_point
  #   model: transactions
  #   given:
  #     - input: ref('raw__transactions')
  #       rows:
  #         - {id: 2, amount: 100, cu

#### run test

In [21]:
%%bash
export DBT_PROJECT_DIR=/usr/app/dbt/dbt_spark_demo_prj
dbt test --log-level info

14:54:10  Running with dbt=1.10.2


14:54:11  Registered adapter: spark=1.9.2


14:54:12  Found 1 seed, 1 model, 1 test, 478 macros, 3 unit tests


14:54:12  


14:54:12  Concurrency: 1 threads (target='spark')


...

14:54:17  


14:54:17  Finished running 1 test, 3 unit tests in 0 hours 0 minutes and 5.02 seconds (5.02s).


14:54:17  


14:54:17  Completed successfully
14:54:17  


14:54:17  Done. PASS=4 WARN=0 ERROR=0 SKIP=0 NO-OP=0 TOTAL=4


In [22]:
!chmod -R a+rw /usr/app

In [23]:
filepath="/usr/app/demo/results/test_end.txt"
!date; echo end test at > $filepath 
!export TZ='Asia/Singapore'; date >> $filepath;

Wed Jul  2 14:54:20 UTC 2025


In [24]:
# assert False, "Stop execution here!"

# Apendix

## check the model built 

### Check the model built using Pyspark (In progress)

In [25]:
# !pip install pyspark==3.3.2

In [26]:
# %%bash
# # For Debian/Ubuntu-based containers:
# apt-get update && apt-get install -y openjdk-11-jdk

# # For Alpine-based containers:
# apk add openjdk11

In [27]:
# !ls /usr/lib/jvm/java-11-openjdk-amd64

In [28]:
# # !. /usr/spark/bin/load-spark-env.sh

# # Typically in Debian/Ubuntu:
# !export JAVA_HOME=$(dirname $(dirname $(readlink -f $(which java))))

# # Or set it explicitly:
# !export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64
# from pyspark.sql import SparkSession

# spark = SparkSession.builder \
#     .appName("dbt_clean") \
#     .config("spark.hadoop.hive.metastore.schema.verification", "true") \
#     .config("spark.sql.catalogImplementation", "hive") \
#     .enableHiveSupport() \
#     .getOrCreate()

# # spark = SparkSession.builder \
# #     .config("hive.metastore.uris", "thrift://dbt-hive-metastore:10000") \
# #     .config("spark.sql.catalogImplementation", "hive") \
# #     .enableHiveSupport() \
# #     .getOrCreate()

# spark.sql("USE default")  # Or your specific database
# spark.sql("SHOW DATABASES").show()
# # spark.sql("select * from transactions").show()
# # # df.toPandas()

In [29]:
# spark.sql("SHOW TABLES IN default").show(truncate=False)

In [30]:
# # Query metastore system tables
# spark.sql("""
#   SELECT TBL_NAME, TBL_TYPE 
#   FROM default.TBLS 
#   JOIN default.DBS ON TBLS.DB_ID = DBS.DB_ID 
#   WHERE DBS.NAME = 'default'
# """).show(truncate=False)

In [31]:
# spark.conf.get("hive.metastore.uris")  # Should return your URI

In [32]:
# spark.conf.get("spark.sql.catalogImplementation")  # Should return "hive"

In [33]:
# from pyspark.sql import SparkSession

# spark.stop()
# spark = SparkSession.builder \
#     .appName("DBTIntegration") \
#     .config("hive.metastore.uris", "thrift://dbt-hive-metastore:9083") \
#     .config("spark.sql.catalogImplementation", "hive") \
#     .config("javax.jdo.option.ConnectionUserName", "dbt") \
#     .config("javax.jdo.option.ConnectionPassword", "dbt") \
#     .enableHiveSupport() \
#     .getOrCreate()
# spark.sql("SHOW TABLES IN default")

In [34]:
# from pyhive import hive
# conn = hive.connect(host=spark_thrift_service_name, port=10000)
# cursor = conn.cursor()
# cursor.execute("SELECT * FROM default.transactions LIMIT 1")
# print(cursor.fetchall())

### Test the built model (using the test model) 

### dbt test

In [35]:
!dbt test --log-level info

14:54:26  Running with dbt=1.10.2


14:54:27  Registered adapter: spark=1.9.2


14:54:28  Found 1 seed, 1 model, 1 test, 478 macros, 3 unit tests
14:54:28  
14:54:28  Concurrency: 1 threads (target='spark')
14:54:28  


14:54:29  1 of 4 START test not_null_transactions_amount ................................. [RUN]


14:54:31  1 of 4 PASS not_null_transactions_amount ....................................... [PASS in 1.69s]
14:54:31  2 of 4 START unit_test transactions::test_currency_sgd ......................... [RUN]


14:54:32  2 of 4 PASS transactions::test_currency_sgd .................................... [PASS in 0.62s]
14:54:32  3 of 4 START unit_test transactions::test_currency_usd ......................... [RUN]


14:54:32  3 of 4 PASS transactions::test_currency_usd .................................... [PASS in 0.51s]
14:54:32  4 of 4 START unit_test transactions::test_inactive ............................. [RUN]


14:54:33  4 of 4 PASS transactions::test_inactive ........................................ [PASS in 0.75s]


14:54:33  
14:54:33  Finished running 1 test, 3 unit tests in 0 hours 0 minutes and 4.64 seconds (4.64s).


14:54:33  
14:54:33  Completed successfully
14:54:33  
14:54:33  Done. PASS=4 WARN=0 ERROR=0 SKIP=0 NO-OP=0 TOTAL=4


In [36]:
# !cat /home/vin/01-prj/stripe/sql-formal-verification/formal_verification_prj_name/tests/unit/test_transactions.yml

In [37]:
# %%bash
# cd /usr/app/dbt
# dbt compile --profiles-dir /usr/app/dbt  --project-dir /usr/app/dbt/dbt_spark_demo_prj
# # dbt run-operation drop_table --args '{"table_name": "default.transactions"}'
# dbt run-operation drop_view --args '{"view_name": "default.my_first_dbt_model"}' --profiles-dir /usr/app/dbt  --project-dir /usr/app/dbt/dbt_spark_demo_prj
# dbt run-operation drop_view --args '{"view_name": "default.my_second_dbt_model"}' --profiles-dir /usr/app/dbt  --project-dir /usr/app/dbt/dbt_spark_demo_prj

<!-- # Todo
## Audit compliance 
### time tracing with time-stamping
1. Add timestamp for loaded data using dbt model
```jinja
SELECT
    *,
    '{{ run_started_at }}'::timestamp AS added_at
FROM {{ ref('my_seed_table') }}
```
### File tracing with source and targets file path, tablename -->


# End of notebook